In [1]:
import pandas as pd
import numpy as np
import logging

In [2]:
"""
BaseStation Class
"""
class BaseStation:
    def __init__(self, id, addr, lat, lng):
        self.id = id
        self.address = addr
        self.lattitude = lat
        self.longitude = lng
        self.num_users = 0
        self.workload = 0
        self.potential_user = 0 # nilai skor potential user
    
    def __str__(self):
        return f"No.{self.id}: {self.addrress}"

In [6]:
"""
Fungsi untuk memuat data BS dari CSV dan menghasilkan daftar BS
"""
def load_base_station(bs_csv_path):
    bs_data = pd.read_csv(bs_csv_path, index_col=0)
    base_stations = []
    for idx, row in bs_data.iterrows():
        bs = BaseStation(id=int(row['id']), addr=row['address'], 
                         lat=float(row['latitude']), 
                         lng=float(row['longitude']))
        base_stations.append(bs)
    return base_stations

In [8]:
bs_csv = 'bs_all.csv'
user_csv = 'data_all.csv'

base_stations = load_base_station(bs_csv)
logging.info(f"Loaded {len(base_stations)} base station")

In [ ]:
"""
Fungsi untuk menggabungkan data transaksi pengguna ke tiap BS
"""
def aggregate_user_data(base_stations, user_csv_path):
    # Mapping dari address ke objek BS
    addr_to_bs = {bs.address: bs for bs in base_stations}
    
    # Memuat data transaksi pengguna
    user_data = pd.read_csv(user_csv_path, index_col=0)
    user_data['start time'] = pd.to_datetime(user_data['start time'])
    user_data['end time'] = pd.todatetime(user_date['end time'])
    
    # Menghitung durasi layanan dalam menit
    user_data['service_time'] = (user_data['end time'] - user_data['start time']).dt.total_seconds() / 60
    
    # Agregasi berdasarkan address: total service_time, jumlah pengguna unik, dan jumlah transaksi
    grouped = user_data.groupby('address').agg({
        'service_time': 'sum',
        'user_id': pd.Series.nunique,
        'start time': 'count'
    }).rename(columns={'user id': 'unique users', 'start time': 'transaction_count'})

    # Input data agregasi ke masing-masing BS
    for bs in base_stations:
        if bs.address in grouped.index:
            row = grouped.loc[bs.address]
            bs.workload = row['service_time']
            bs.num_users = row['unique_users']
        else:
            bs.workload = 0
            bs.num_users = 0
    return base_stations    

In [ ]:
"""
Menghitung skor potential user pada tiap BS dengan 
menggabungkan jumlah pengguna unik dan total service_time 
(dengan normalisasi sederhana).
"""

def compute_potential_user(base_stations):
    user_counts = np.array([bas.num_users for bs in base_stations])
    workloads = np.array([bs.workload for bs in base_stations])
    
    # Normalisasi, antara 0 dan 1
    if user_counts.max() > user_counts.min():
        norm_users = (user_counts - user_counts.min() / user_counts.max() - user_counts.min())
    else:
        norm_users = user_counts
        
    if workloads.max() > workloads.min():
        norm_workloads = (workloads - workloads.min()) / (workloads.max() - workloads.min())
    else:
        norm_workloads = workloads
    
    # Kombinasi sederhana dengan memeakai bobot yang sama
    for i, bs in enumerate(base_stations):
        bs.potential_user = 0.5 * norm_users[i] + 0.5 * norm_workloads[i]
    return base_stations

In [ ]:
"""
Menyaring BS dengan skor potential_user di bawah threshold
"""

def filte_base_station(base_stations, threshold=0.1):
    filtered = [bs for bs in base_stations if bs.potential_user >= threshold]
    logging.info(f"Filtered out {len(base_stations) - len(filtered)} base stations below threshold {threshold}")
    return filtered